# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

## Load Spark and Data

In [75]:
# start spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_project').getOrCreate()

In [76]:
# read in the input csv file.
data = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [77]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [78]:
for r in data.head(1)[0]:
    print(r)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


## Clear and format data

In [95]:
# format the columns
cleaned_data = data.select([
    data['Cruise_line'].alias('line'),
    data['Age'].alias('age'),
    (data['Tonnage']*1000).cast('integer').alias('tonnage'),
    (data['passengers']*100).cast('integer').alias('passengers'),
    (data['length']*100).cast('integer').alias('length'),
    (data['cabins']*100).cast('integer').alias('cabins'),
    data['passenger_density'],
    (data['crew']*100).cast('integer').alias('crew')
])
cleaned_data.show()

+--------+---+-------+----------+------+------+-----------------+----+
|    line|age|tonnage|passengers|length|cabins|passenger_density|crew|
+--------+---+-------+----------+------+------+-----------------+----+
| Azamara|  6|  30276|       694|   594|   355|            42.64| 355|
| Azamara|  6|  30276|       694|   594|   355|            42.64| 355|
|Carnival| 26|  47262|      1486|   722|   743|             31.8| 670|
|Carnival| 11| 110000|      2974|   952|  1488|            36.99|1910|
|Carnival| 17| 101353|      2642|   892|  1321|            38.36|1000|
|Carnival| 22|  70367|      2052|   855|  1019|            34.29| 919|
|Carnival| 15|  70367|      2052|   855|  1019|            34.29| 919|
|Carnival| 23|  70367|      2056|   855|  1022|            34.23| 919|
|Carnival| 19|  70367|      2052|   855|  1019|            34.29| 919|
|Carnival|  6| 110238|      3700|   951|  1487|            29.79|1150|
|Carnival| 10| 110000|      2974|   951|  1487|            36.99|1160|
|Carni

In [96]:
cleaned_data.printSchema()

root
 |-- line: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- tonnage: integer (nullable = true)
 |-- passengers: integer (nullable = true)
 |-- length: integer (nullable = true)
 |-- cabins: integer (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: integer (nullable = true)



In [97]:
# list number of NANs or NULLs in each column
from pyspark.sql.functions import count, when, isnan, col
cleaned_data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in cleaned_data.columns]).show()

+----+---+-------+----------+------+------+-----------------+----+
|line|age|tonnage|passengers|length|cabins|passenger_density|crew|
+----+---+-------+----------+------+------+-----------------+----+
|   0|  0|      0|         0|     0|     0|                0|   0|
+----+---+-------+----------+------+------+-----------------+----+



Nothing to clean, no nulls. Data looks good.

## Prepare data for MLlib

In [98]:
# convert line string to int 
from pyspark.ml.feature import StringIndexer
string_indexer = StringIndexer(inputCol="line", outputCol="line_index")
prepared_data = indexer.fit(cleaned_data).transform(cleaned_data)
prepared_data = prepared_data.select([prepared_data['line_index'].alias('line'),'age','tonnage','passengers','length','cabins','passenger_density'])
prepared_data.show()

+----+---+-------+----------+------+------+-----------------+
|line|age|tonnage|passengers|length|cabins|passenger_density|
+----+---+-------+----------+------+------+-----------------+
|16.0|  6|  30276|       694|   594|   355|            42.64|
|16.0|  6|  30276|       694|   594|   355|            42.64|
| 1.0| 26|  47262|      1486|   722|   743|             31.8|
| 1.0| 11| 110000|      2974|   952|  1488|            36.99|
| 1.0| 17| 101353|      2642|   892|  1321|            38.36|
| 1.0| 22|  70367|      2052|   855|  1019|            34.29|
| 1.0| 15|  70367|      2052|   855|  1019|            34.29|
| 1.0| 23|  70367|      2056|   855|  1022|            34.23|
| 1.0| 19|  70367|      2052|   855|  1019|            34.29|
| 1.0|  6| 110238|      3700|   951|  1487|            29.79|
| 1.0| 10| 110000|      2974|   951|  1487|            36.99|
| 1.0| 28|  46052|      1452|   727|   726|            31.72|
| 1.0| 18|  70367|      2052|   855|  1019|            34.29|
| 1.0| 1

In [99]:
# import helpers
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [100]:
prepared_data.printSchema()

root
 |-- line: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- tonnage: integer (nullable = true)
 |-- passengers: integer (nullable = true)
 |-- length: integer (nullable = true)
 |-- cabins: integer (nullable = true)
 |-- passenger_density: double (nullable = true)



In [102]:
# create and configure the vector assembler with the desired 'features' columns
assembler = VectorAssembler(
    inputCols=['line','age','tonnage','passengers','length','cabins','passenger_density'],
    outputCol='features')

# assemble the 'features'
output = assembler.transform(prepared_data)
output..show()

+----+---+-------+----------+------+------+-----------------+--------------------+
|line|age|tonnage|passengers|length|cabins|passenger_density|            features|
+----+---+-------+----------+------+------+-----------------+--------------------+
|16.0|  6|  30276|       694|   594|   355|            42.64|[16.0,6.0,30276.0...|
|16.0|  6|  30276|       694|   594|   355|            42.64|[16.0,6.0,30276.0...|
| 1.0| 26|  47262|      1486|   722|   743|             31.8|[1.0,26.0,47262.0...|
| 1.0| 11| 110000|      2974|   952|  1488|            36.99|[1.0,11.0,110000....|
| 1.0| 17| 101353|      2642|   892|  1321|            38.36|[1.0,17.0,101353....|
| 1.0| 22|  70367|      2052|   855|  1019|            34.29|[1.0,22.0,70367.0...|
| 1.0| 15|  70367|      2052|   855|  1019|            34.29|[1.0,15.0,70367.0...|
| 1.0| 23|  70367|      2056|   855|  1022|            34.23|[1.0,23.0,70367.0...|
| 1.0| 19|  70367|      2052|   855|  1019|            34.29|[1.0,19.0,70367.0...|
| 1.